<a href="https://colab.research.google.com/github/bkvkrll/PyTorch/blob/main/lesson3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:
1. Создать Dataset для загрузки данных
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [19]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [20]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHES = 10
LR = 0.01

In [21]:
class MyOwnDataset(Dataset):
  def __init__(self, *init_datasets):
    assert all(init_datasets[0].size(0) == init_dataset.size(0) for init_dataset in init_datasets), "Несоотвутствует размерность среди dataset"
    self._base_datasets = init_datasets
  
  def __len__(self):
    return self._base_datasets[0].size(0)
  
  def __getitem__(self, idx):
      return tuple(base_dataset[idx] for base_dataset in self._base_datasets)

In [22]:
class Net(nn.Module):
  def __init__(self) -> None:
      super(Net, self).__init__()
      self.block_1 = nn.Sequential(
          nn.Linear(in_features=8, out_features=100, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_2 = nn.Sequential(
          nn.Linear(in_features=100, out_features=100, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_3 = nn.Sequential(
          nn.Linear(in_features=100, out_features=60, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(60),
          nn.ReLU())
      self.block_4 = nn.Sequential(
          nn.Linear(in_features=60, out_features=30),
          nn.Dropout(0.2),
          nn.BatchNorm1d(30),
          nn.ReLU())
      self.predict = nn.Sequential(
          nn.Linear(in_features=30, out_features=1, bias=True),
          nn.BatchNorm1d(1),
          nn.ReLU())
  
  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    out = self.block_4(out)
    out = self.predict(out)
    return out[:, 0]

In [23]:
def train_loop(train_loader, test_loader, net, optimizer):
  loss_fn = nn.MSELoss()
  best_acc = {'train': None, 'test': None}
  net.train()
  for epoch in range(EPOCHES):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)

        # обнуляем градиент
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 150 == 0 or (i + 1) == len(train_loader):    # печатаем каждые 150 mini-batches
            net.eval()

            test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
            for y, (out_test, lbl_test) in enumerate(test_loader):
                test_outputs = net(out_test)
                test_loss += loss_fn(test_outputs, lbl_test)
                test_running_total += len(lbl_test)
            
            res_loss_train = running_loss / running_items
            res_loss_test = test_loss / test_running_total
            
            if best_acc['train'] is None or res_loss_train < best_acc['train']:
              best_acc['train'] = res_loss_train
            
            if best_acc['test'] is None or res_loss_test < best_acc['test']:
              best_acc['test'] = res_loss_train

            print(f'Epoch [{epoch + 1}/{EPOCHES}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {res_loss_train:.3f}. '\
                  f'Test acc: {res_loss_test:.3f}.')
            
            running_loss, running_items = 0.0, 0.0
            net.train()
  print(f"Best acc train: {best_acc['train']:.3f}. Best acc test: {best_acc['test']:.3f}")
  print('Training is finished!')

In [24]:
california_housing = fetch_california_housing()
# Разделим на тестовые и тренеровочные данные
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target, test_size=0.25, random_state=13)

In [25]:
scale = StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [26]:
train_xt = torch.from_numpy(X_train_s.astype(np.float32)).to(DEVICE)
train_yt = torch.from_numpy(y_train.astype(np.float32)).to(DEVICE)

test_xt = torch.from_numpy(X_test_s.astype(np.float32)).to(DEVICE)
test_yt = torch.from_numpy(y_test.astype(np.float32)).to(DEVICE)

In [27]:
train_dataset = MyOwnDataset(train_xt, train_yt)
test_dataset = MyOwnDataset(test_xt, test_yt)

In [28]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)

**Adam**

In [29]:
net = Net().to(DEVICE)
optimizer = Adam(net.parameters(), lr=LR)

In [30]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.090. Test acc: 0.073.
Epoch [1/10]. Step [151/241]. Loss: 0.026. Test acc: 0.011.
Epoch [1/10]. Step [241/241]. Loss: 0.010. Test acc: 0.013.
Epoch [2/10]. Step [1/241]. Loss: 0.011. Test acc: 0.013.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.008.
Epoch [2/10]. Step [241/241]. Loss: 0.009. Test acc: 0.011.
Epoch [3/10]. Step [1/241]. Loss: 0.008. Test acc: 0.011.
Epoch [3/10]. Step [151/241]. Loss: 0.008. Test acc: 0.006.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.012.
Epoch [4/10]. Step [1/241]. Loss: 0.007. Test acc: 0.013.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.007.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.009.
Epoch [5/10]. Step [1/241]. Loss: 0.008. Test acc: 0.008.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.013.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [6/10]. Step [1/241]. Loss: 0.007. Test acc: 0.007.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

**RMSProp**

In [31]:
net = Net().to(DEVICE)
optimizer = RMSprop(net.parameters(), lr=LR)

In [32]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.073. Test acc: 0.089.
Epoch [1/10]. Step [151/241]. Loss: 0.015. Test acc: 0.012.
Epoch [1/10]. Step [241/241]. Loss: 0.009. Test acc: 0.007.
Epoch [2/10]. Step [1/241]. Loss: 0.006. Test acc: 0.007.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.006.
Epoch [2/10]. Step [241/241]. Loss: 0.007. Test acc: 0.006.
Epoch [3/10]. Step [1/241]. Loss: 0.009. Test acc: 0.007.
Epoch [3/10]. Step [151/241]. Loss: 0.007. Test acc: 0.009.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [4/10]. Step [1/241]. Loss: 0.011. Test acc: 0.008.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.006.
Epoch [4/10]. Step [241/241]. Loss: 0.006. Test acc: 0.009.
Epoch [5/10]. Step [1/241]. Loss: 0.006. Test acc: 0.008.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.009.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [6/10]. Step [1/241]. Loss: 0.004. Test acc: 0.007.
Epoch [6/10]. Step [151/241]. Loss: 0.006. Test acc:

**SGD**

In [33]:
net = Net().to(DEVICE)
optimizer = SGD(net.parameters(), lr=LR)

In [34]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.078. Test acc: 0.082.
Epoch [1/10]. Step [151/241]. Loss: 0.026. Test acc: 0.011.
Epoch [1/10]. Step [241/241]. Loss: 0.012. Test acc: 0.010.
Epoch [2/10]. Step [1/241]. Loss: 0.007. Test acc: 0.010.
Epoch [2/10]. Step [151/241]. Loss: 0.011. Test acc: 0.009.
Epoch [2/10]. Step [241/241]. Loss: 0.010. Test acc: 0.009.
Epoch [3/10]. Step [1/241]. Loss: 0.011. Test acc: 0.009.
Epoch [3/10]. Step [151/241]. Loss: 0.009. Test acc: 0.007.
Epoch [3/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [4/10]. Step [1/241]. Loss: 0.007. Test acc: 0.007.
Epoch [4/10]. Step [151/241]. Loss: 0.008. Test acc: 0.007.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [5/10]. Step [1/241]. Loss: 0.005. Test acc: 0.007.
Epoch [5/10]. Step [151/241]. Loss: 0.008. Test acc: 0.007.
Epoch [5/10]. Step [241/241]. Loss: 0.008. Test acc: 0.008.
Epoch [6/10]. Step [1/241]. Loss: 0.007. Test acc: 0.008.
Epoch [6/10]. Step [151/241]. Loss: 0.008. Test acc:

Скорость обучения трех вариантов меняется не значительно, лучшие значения на тестовой выборке показал оптимизатор RMSProp